In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。
load_dotenv(find_dotenv())

# # 如果你需要通过代理端口访问，还需要做如下配置
# os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
# os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'

True

In [2]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    # A list of coversation
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ]
)

In [3]:
completion

ChatCompletion(id='chatcmpl-9GWAtAv9lcvzMbPPpX0GWeTUtZSYv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1713723955, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=9, prompt_tokens=19, total_tokens=28))

In [4]:
completion.choices[0].message.content

'Hello! How can I assist you today?'

In [5]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


def get_completion(prompt, model="gpt-3.5-turbo", temperature = 0):

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"


In [22]:
answer = get_completion("tell me how to install linux system on an rog zephyrus")
print(answer)


To install a Linux system on an ROG Zephyrus laptop, you can follow these steps:

1. Backup your data: Before installing a new operating system, it is important to backup all your important data to prevent any loss.

2. Download a Linux distribution: Choose a Linux distribution that you want to install on your laptop. Popular choices include Ubuntu, Fedora, and Linux Mint.

3. Create a bootable USB drive: Download the ISO file of the Linux distribution you chose and create a bootable USB drive using a tool like Rufus or Etcher.

4. Disable Secure Boot: Access the BIOS settings of your ROG Zephyrus laptop by pressing the appropriate key during startup (usually F2 or Del). Disable Secure Boot to allow the installation of a non-Windows operating system.

5. Boot from the USB drive: Insert the bootable USB drive into your laptop and restart it. Press the appropriate key to access the boot menu and select the USB drive as the boot device.

6. Install Linux: Follow the on-screen instructions

# embedding

In [8]:
def openai_embedding(text: str, model: str=None):
    # api_key=os.environ['OPENAI_API_KEY']
    # client = OpenAI(api_key=api_key)

    # embedding model：'text-embedding-3-small', 'text-embedding-3-large', 'text-embedding-ada-002'
    if model == None:
        model="text-embedding-3-small"

    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response

response = openai_embedding(text='要生成 embedding 的输入文本，字符串形式。')

In [9]:
response.data

[Embedding(embedding=[0.03884002938866615, 0.013516489416360855, -0.0024250170681625605, -0.01655769906938076, 0.024130908772349358, -0.017382603138685226, 0.04206013306975365, 0.011498954147100449, -0.028245486319065094, -0.00674333656206727, 0.0011976007372140884, 0.014013418927788734, -0.023097295314073563, 0.01580236665904522, -0.005903525277972221, 0.013764954172074795, -0.010624358430504799, -0.010823129676282406, -0.01147907692939043, 0.02377311885356903, 0.023673733696341515, -0.008934796787798405, -0.00743406917899847, 0.017124198377132416, 0.016855856403708458, -0.02202392742037773, 0.020990312099456787, -0.009993257001042366, 0.04245767742395401, -0.001410038210451603, -0.08571044355630875, -0.040986765176057816, 0.01652788370847702, -0.03925745189189911, 0.012810848653316498, 0.0416228361427784, -0.008224187418818474, -0.013238208368420601, 0.021666137501597404, 0.015991199761629105, 0.004869911354035139, 0.012999682687222958, -0.0002458249218761921, 0.025502434000372887, -

In [10]:
len(response.data[0].embedding)

1536

In [11]:
response.model

'text-embedding-3-small'

In [12]:
response.usage

Usage(prompt_tokens=12, total_tokens=12)

# database

In [2]:
file_paths = []
folder_path = 'data'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

['data/pdf/rog_zephyrus_manual.pdf', 'data/markdowm/Guide_to_linux_for_ASUS_ROG_laptops.md', 'data/markdowm/ubuntu-on-asus-g14.md']


In [3]:
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# append different kinds of loaders to a list for further use
loaders = []

for file_path in file_paths:

    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    elif file_type == 'md':
        loaders.append(UnstructuredMarkdownLoader(file_path))


In [4]:
loaders

In [5]:
# load the files to texts
texts = []

for loader in loaders: texts.extend(loader.load())

In [6]:
texts[:10]

[Document(page_content='More info:\nE20733\nFirst Edition / February 2023\n', metadata={'source': 'data/pdf/rog_zephyrus_manual.pdf', 'file_path': 'data/pdf/rog_zephyrus_manual.pdf', 'page': 0, 'total_pages': 104, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 18.1 (Windows)', 'producer': 'Adobe PDF Library 17.0', 'creationDate': "D:20230131190002+08'00'", 'modDate': "D:20230131190108+08'00'", 'trapped': '', 'encryption': 'Standard V5 R6 256-bit AES'}),
 Document(page_content='2\nNotebook PC E-Manual\nCOPYRIGHT INFORMATION\nNo part of this manual, including the products and software described in it, may be reproduced, \ntransmitted, transcribed, stored in a retrieval system, or translated into any language in any form or by \nany means, except documentation kept by the purchaser for backup purposes, without the express \nwritten permission of ASUSTeK COMPUTER INC. (“ASUS”).\nASUS PROVIDES THIS MANUAL “AS IS” WITHOUT WARRANTY OF

In [7]:
text = texts[1]
print(f"type of element：{type(text)}.", 
    f"metadata about the text：{text.metadata}", 
    f"content of the text:\n{text.page_content[0:]}", 
    sep="\n------\n")

type of element：<class 'langchain_core.documents.base.Document'>.
------
metadata about the text：{'source': 'data/pdf/rog_zephyrus_manual.pdf', 'file_path': 'data/pdf/rog_zephyrus_manual.pdf', 'page': 1, 'total_pages': 104, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 18.1 (Windows)', 'producer': 'Adobe PDF Library 17.0', 'creationDate': "D:20230131190002+08'00'", 'modDate': "D:20230131190108+08'00'", 'trapped': '', 'encryption': 'Standard V5 R6 256-bit AES'}
------
content of the text:
2
Notebook PC E-Manual
COPYRIGHT INFORMATION
No part of this manual, including the products and software described in it, may be reproduced, 
transmitted, transcribed, stored in a retrieval system, or translated into any language in any form or by 
any means, except documentation kept by the purchaser for backup purposes, without the express 
written permission of ASUSTeK COMPUTER INC. (“ASUS”).
ASUS PROVIDES THIS MANUAL “AS IS” WITHOUT WARRAN

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split the ducoments
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100)

split_docs = text_splitter.split_documents(texts)


In [9]:
len(split_docs[7].page_content)

897

In [10]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings() 

In [11]:
persist_directory = 'data_base/vector_db/chroma'
!rm -rf 'data_base/vector_db/chroma'

In [12]:
from langchain.vectorstores.chroma import Chroma

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding,
    persist_directory=persist_directory
)


In [13]:
vectordb.persist()

In [14]:
print(f"number of vectors in db：{vectordb._collection.count()}")

number of vectors in db：154


# vector search

## cosine similarity

In [15]:
question = "tell me how to install linux system on an rog zephyrus"
sim_docs = vectordb.similarity_search(question,k=3)
print(f"number of search results: {len(sim_docs)}")


number of search results: 3


In [16]:
for i, sim_doc in enumerate(sim_docs):
    print(f"the {i}th search result: \n{sim_doc.page_content}", 
          end="\n-----------------------------\n")


the 0th search result: 
Ubuntu on Asus ROG Zephyrus G14 2021 (Setup guide)

Here is a way to do a robust install of Ubuntu (+ optional Windows 11 dual boot and LUKS encryption) on an Asus laptop, with minimal usable hardware support, without a significant amount of tinkering that may break in future or require frequent technical attention.

In summary, the key thing is to have an up to date kernel, which usually means disabling secure-boot and installing the latest stable (6.0+) using mainline or xanmod, and as well as making sure the latest nvidia driver and dkms is installed.

Specs:

Model Asus G14 2021 (GA401QC)

AMD R7 5800 8 core 16 thread (onboard Radeon graphics)

NVIDIA RTX 3050 4GB (60W +15W boost)

40GB RAM (8GB soldered + 32GB stick added)

2TB SSD

14inch 1920x1080 Display @144Hz

~Mediatek Wifi 6 card~ Intel AX210 Wifi 6 card (Replaced built-in Mediatek card)

OS Install:
-----------------------------
the 1th search result: 
Guide to linux for ASUS ROG laptops

I purchase

## MMR

In [17]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)
for i, sim_doc in enumerate(mmr_docs):
    print(f"The {i}th content of MMR: \n{sim_doc.page_content}", end="\n--------------\n")


The 0th content of MMR: 
Ubuntu on Asus ROG Zephyrus G14 2021 (Setup guide)

Here is a way to do a robust install of Ubuntu (+ optional Windows 11 dual boot and LUKS encryption) on an Asus laptop, with minimal usable hardware support, without a significant amount of tinkering that may break in future or require frequent technical attention.

In summary, the key thing is to have an up to date kernel, which usually means disabling secure-boot and installing the latest stable (6.0+) using mainline or xanmod, and as well as making sure the latest nvidia driver and dkms is installed.

Specs:

Model Asus G14 2021 (GA401QC)

AMD R7 5800 8 core 16 thread (onboard Radeon graphics)

NVIDIA RTX 3050 4GB (60W +15W boost)

40GB RAM (8GB soldered + 32GB stick added)

2TB SSD

14inch 1920x1080 Display @144Hz

~Mediatek Wifi 6 card~ Intel AX210 Wifi 6 card (Replaced built-in Mediatek card)

OS Install:
--------------
The 1th content of MMR: 
for arch-linux or manjaro

open software centre

enable AUR


# openai with longchain

In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

llm.invoke("hello")


AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-cd890f95-2e8e-4c42-9e68-39fee361d1bb-0')